In [8]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

# 라이브러리 선언
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.retrievers import BM25Retriever, EnsembleRetriever

In [ ]:
# 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # 1536 차원

# DB 경로 설정
persist_directory = "../.db/faiss"

vectorstore = FAISS.load_local(
    persist_directory, 
    embeddings, 
    allow_dangerous_deserialization=True)

print(f"FAISS 인덱스 크기: {vectorstore.index.ntotal}개의 문서")

In [2]:
def pretty_print_with_score(results_with_score):
    if results_with_score:
        for i, (doc, score) in enumerate(results_with_score, 1):
            print(f"[결과 {i}] 유사도: {score}") # 낮을수록 더 유사함
            print(f"{doc.page_content}")
            # print(f"메타데이터: {doc.metadata}")  # 모든 메타데이터 출력
            try:
                print(f"- 상품상태: {doc.metadata.get('GOODS_STAT_SCT_NM', '정보 없음')}")
                print(f"- 브랜드명: {doc.metadata.get('BRND_NM', '정보 없음')}")
                print(f"- 상품명: {doc.metadata.get('GOODS_NM', '정보 없음')}")
                print(f"- 품목정보: {doc.metadata.get('ARTC_INFO', '정보 없음')}")
                print(f"- 카테고리: {doc.metadata.get('CATEGORY_NMS', '정보 없음')}")
                # print(f"- 판매가: {format(int(doc.metadata.get('SALE_PRC', 0)), ',')}원")
                # print(f"- 할인가: {format(int(doc.metadata.get('DSCNT_SALE_PRC', 0)), ',')}원")
                print(f"- 최대혜택가: {format(int(doc.metadata.get('MAX_BENEFIT_PRICE', 0)), ',')}원")
                print(f"- 카드할인율: {doc.metadata.get('CARD_DC_RATE', '0')}%")
                print(f"- 할인카드: {doc.metadata.get('CARD_DC_NAME_LIST', '정보 없음')}")
                print(f"- 주요 특징 및 기능:")
                feature_values = doc.metadata['OPT_VAL_DESC'].split(',')
                feature_titles = doc.metadata['OPT_DISP_NM'].split(',')
                for i, (title, value) in enumerate(zip(feature_titles, feature_values)):
                    print(f"  - {title} : {value}")
                    if i < 3: break
                print(f"🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo={doc.metadata.get('GOODS_NO', '정보 없음')}")
            except Exception as e:
                print(f"메타데이터 처리 중 오류 발생: {str(e)}")
            print("-" * 60)
    else:
        print("검색 결과가 없습니다.")
        print("다른 검색어로 다시 시도해보시거나, 검색어를 더 구체적으로 입력해주세요.")

In [16]:
def pretty_print(query, result):
    print(f"'{query}'와 유사한 상품 검색 결과:")
    print("-" * 60)

    if result:
        for i, (doc) in enumerate(result, 1):
            print(f"[결과 {i}]")
            print(f"{doc.page_content}")
            # print(f"메타데이터: {doc.metadata}")  # 모든 메타데이터 출력
            try:
                print(f"- 상품상태: {doc.metadata.get('GOODS_STAT_SCT_NM', '정보 없음')}")
                print(f"- 브랜드명: {doc.metadata.get('BRND_NM', '정보 없음')}")
                print(f"- 상품명: {doc.metadata.get('GOODS_NM', '정보 없음')}")
                print(f"- 품목정보: {doc.metadata.get('ARTC_INFO', '정보 없음')}")
                print(f"- 카테고리: {doc.metadata.get('CATEGORY_NMS', '정보 없음')}")
                # print(f"- 판매가: {format(int(doc.metadata.get('SALE_PRC', 0)), ',')}원")
                # print(f"- 할인가: {format(int(doc.metadata.get('DSCNT_SALE_PRC', 0)), ',')}원")
                print(f"- 최대혜택가: {format(int(doc.metadata.get('MAX_BENEFIT_PRICE', 0)), ',')}원")
                print(f"- 판매량: {format(int(doc.metadata.get('SALE_QTY', 0)), ',')}")
                print(f"- 세일즈유닛: {format(int(doc.metadata.get('SALES_UNIT', 0)), ',')}")
                # print(f"- 카드할인율: {doc.metadata.get('CARD_DC_RATE', '0')}%")
                # print(f"- 할인카드: {doc.metadata.get('CARD_DC_NAME_LIST', '정보 없음')}")
                print(f"- 주요 특징 및 기능:")
                feature_values = doc.metadata['OPT_VAL_DESC'].split(',')
                feature_titles = doc.metadata['OPT_DISP_NM'].split(',')
                for i, (title, value) in enumerate(zip(feature_titles, feature_values)):
                    print(f"  - {title} : {value}")
                    if i < 3: break
                print(f"🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo={doc.metadata.get('GOODS_NO', '정보 없음')}")
            except Exception as e:
                print(f"메타데이터 처리 중 오류 발생: {str(e)}")
            print("-" * 60)
    else:
        print("검색 결과가 없습니다.")
        print("다른 검색어로 다시 시도해보시거나, 검색어를 더 구체적으로 입력해주세요.")

In [9]:
def extract_documents_from_faiss(faiss_db):
    """FAISS DB에서 Document 객체들을 추출"""
    documents = []
    
    # docstore에서 모든 Document 객체 추출
    for doc_id in faiss_db.docstore._dict.keys():
        doc = faiss_db.docstore.search(doc_id)
        # 이미 Document 객체이므로 그대로 사용
        documents.append(doc)
    
    return documents

In [21]:
import re

def classify_query_type(query: str) -> str:
    """
    검색어가 자연어인지 키워드인지 간단한 규칙 기반으로 판단합니다.
    """
    query = query.strip()

    # 1. 길이 기반 판단
    words = query.split()
    num_words = len(words)
    if num_words > 5: # 5단어 초과면 자연어일 가능성 높음
        return "natural_language"

    # 2. 불용어 및 조사/어미 패턴 확인 (한국어 기준)
    # 간단한 불용어 리스트와 문장 종결 어미, 조사 패턴
    stopwords_korean = ["은", "는", "이", "가", "을", "를", "에", "에서", "와", "과", "로", "으로", "도", "만", "좀", "요", "입니다", "있나요", "해주세요", "추천해주세요", "어떤", "무엇"]
    
    for sw in stopwords_korean:
        if sw in query:
            # 불용어가 포함되어 있으면 자연어일 가능성 높음 (단, 짧은 쿼리 예외 처리 필요)
            if num_words > 2: # 2단어 초과 쿼리에 불용어 포함 시 자연어
                return "natural_language"

    # 3. 질문/명령형 어미 확인 (더 명확한 자연어 신호)
    if re.search(r'(주세요|해줘|추천|있나|있나요|떤가|떤가요|을까|을까요|뭔가요)\s*$', query):
        return "natural_language"
    
    # 4. 명확한 키워드 패턴 (예: 브랜드 + 모델명)
    # 실제 시스템에서는 상품명 DB와 비교하여 판단할 수 있습니다.
    # 여기서는 간단히 '브랜드 + 모델명' 패턴을 가정
    if re.match(r'^(삼성|LG|애플|다이슨)\s+.*(TV|냉장고|폰|청소기|세탁기|에어컨)', query, re.IGNORECASE):
         if num_words <= 4: # 짧은 키워드 조합일 경우
             return "keyword"


    # 위 조건에 해당하지 않으면 기본적으로 키워드로 간주 (더 짧고 명료한 쿼리)
    return "keyword"

In [26]:

# faiss retriever를 초기화합니다.
faiss_retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# bm25 retriever를 초기화합니다.
documents = extract_documents_from_faiss(vectorstore)
bm25_retriever = BM25Retriever.from_documents(documents)
bm25_retriever.k = 2  # BM25Retriever의 검색 결과 개수를 1로 설정합니다.

# 앙상블 retriever를 초기화합니다.
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.7, 0.3],
    search_kwargs={"k": 2}
)

In [ ]:
# 검색 결과 문서를 가져옵니다.
query = "원룸형 냉장고"

print(f"쿼리 타입: {classify_query_type(query)}") # keyword

# 검색기 초기화
# ensemble_result = ensemble_retriever.invoke(query)
bm25_result = bm25_retriever.invoke(query)
# faiss_result = faiss_retriever.invoke(query)

# 가져온 문서를 출력합니다.
# print("[Ensemble Retriever]")
# pretty_print(query, ensemble_result)
# print()

print("[BM25 Retriever]")
pretty_print(query, bm25_result)
print()

# print("[FAISS Retriever]")
# pretty_print(query, faiss_result)
# print()

원룸형 냉장고: keyword
[BM25 Retriever]
'원룸형 냉장고'와 유사한 상품 검색 결과:
------------------------------------------------------------
[결과 1]
리페르의 양문형냉장고, 리페르 프리미엄 냉장고 SK4260. 카테고리: 냉장고·주방가전 > 냉장고 > 3·4도어냉장고. 가격 정보: 판매가 2,470,000원. 가전제품입니다. 해시태그: #독일가전#독일냉장고#독일명품#명품가전#명품냉장고#빌트인냉장고#연예인냉장고#홈바.
- 상품상태: 정상상품
- 브랜드명: 리페르
- 상품명: 리페르 프리미엄 냉장고 SK4260
- 품목정보: 냉장고>양문형냉장고
- 카테고리: 냉장고·주방가전>냉장고>3·4도어냉장고
- 최대혜택가: 2,470,000원
- 판매량: 0
- 세일즈유닛: 0
- 주요 특징 및 기능:
  -  : 
🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo=0019647959
------------------------------------------------------------
[결과 2]
리페르의 양문형냉장고, 리페르 프리미엄 냉장고 SK4210. 카테고리: 냉장고·주방가전 > 냉장고 > 3·4도어냉장고. 가격 정보: 판매가 2,697,000원. 가전제품입니다. 해시태그: #독일가전#독일냉장고#독일명품#명품가전#명품냉장고#빌트인냉장고#연예인냉장고#홈바.
- 상품상태: 정상상품
- 브랜드명: 리페르
- 상품명: 리페르 프리미엄 냉장고 SK4210
- 품목정보: 냉장고>양문형냉장고
- 카테고리: 냉장고·주방가전>냉장고>3·4도어냉장고
- 최대혜택가: 2,697,000원
- 판매량: 0
- 세일즈유닛: 0
- 주요 특징 및 기능:
  -  : 
🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo=0019647961
-------------------